In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras import initializers

from keras.datasets import mnist

import matplotlib.pyplot as plt

Using TensorFlow backend.
C:\Users\Taral\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Taral\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Taral\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Taral\Anaconda3\envs\te

# Music generation using Generative Adverserial Networks


In [2]:
def discriminator_model(dropout=0.5):
    model = Sequential()
    model.add(Dense(100, input_dim=100, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(dropout))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [3]:
def generator_model():
    model = Sequential()
    model.add(Dense(256, input_dim= 100, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dense(100, activation='tanh'))
    opt = Adam(lr=0.00005)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [4]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)
    print("Network: ", network_input.shape, network_input[0].shape, network_input[1].shape)
    discriminator = discriminator_model()
    generator = generator_model()
    discriminator.trainable = False
    gan_input = Input(shape=(100,))

    x = generator(gan_input)

    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=gan_output)
    opt = Adam(lr=0.0001)
    gan.compile(loss='binary_crossentropy', optimizer=opt)
                                  

    batch_size = 32
    n_steps = 2000


    noise_input = np.random.uniform(-1, 1, size=[16, 100])
    for step in range(n_steps):

        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        batch = network_input[np.random.randint(0, network_input.shape[0], size=batch_size)].reshape(batch_size, 100)

        gen_output = generator.predict(noise)
        

        X = np.concatenate([batch, gen_output])

        y_D = np.zeros(2*batch_size)
        y_D[:batch_size] = 1

        discriminator.trainable = True
        loss_D = discriminator.train_on_batch(X, y_D)

        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        y_G = np.ones(batch_size)
        discriminator.trainable = False
        loss_G = gan.train_on_batch(noise, y_G)

    generator.save("generator300.h5")  
    
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

#     for file in glob.glob("Marshmello1/*/*.mid"):
    for file in glob.glob("bollywood/**.mid"):
        midi = converter.parse(file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    x_note_to_int = dict((note_, number) for number, note_ in enumerate(notes))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100


    pitchnames = sorted(set(item for item in notes))

    
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)


    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)
    

if __name__ == '__main__':
    train_network()

Network:  (141126, 100, 1) (100, 1) (100, 1)
